# Clase Práctica #11 (Compilación)

En esta clase estaremos implementando un **generador de parsers LR(1)**. Procederemos de forma análoga a la clase anterior: primero implementaremos el mecanismo de construcción del **autómata LR(1)** y posteriormente heredaremos de **Shift-Reduce parser** para llenar la tabla **Action-Goto** de forma acorde.

Comencemos por importar la clase `Grammar`.

In [ ]:
from cmp.pycompiler import Grammar

Trabajaremos sobre el lenguaje de expresiones que usamos en la última conferencia.

In [ ]:
G = Grammar()
E = G.NonTerminal('E', True)
A = G.NonTerminal('A')
equal, plus, num = G.Terminals('= + int')

E %=  A + equal + A | num
A %= num + plus + A | num

print(G)

## Items

Usaremos la implementación de `Item` provista en `cmp.pycompiler` para modelar los items LR(1). Esta vez haremos uso del parámetro `lookaheads` que se especifica en el constructor de la clase `Item` para almacenar justamente los `lookaheads` de los items LR(1).

In [ ]:
from cmp.pycompiler import Item

item = Item(E.productions[0], 0, lookaheads=[G.EOF, plus])
print('item:', item)

Se incluyó la función `Preview` a cada item. Esta devuelve todas las posibles cadenas que resultan de concatenar _"lo que queda por leer del item tras saltarse `skip=1` símbolos"_ con los posibles lookaheads. Esta función nos será útil, pues sabemos que el lookahead de los items LR(1) se obtiene de calcular el `first` de estas cadenas.

In [ ]:
for preview in item.Preview():
    print('item.Preview:', preview)

## Clausura de un conjunto de items LR(1)

Como primer paso para calcular la clausura, implementaremos la función `expand`. Esta recibe un item LR(1) y devuelve el conjunto de items que sugiere incluir (directamente) debido a la presencia de un `.` delante de un no terminal.

> expand("$Y \to \alpha . X \delta, c$") = { "$X \to . \beta, b$" | $b \in First(\delta c)$ }

In [ ]:
from cmp.utils import ContainerSet
from cmp.tools.parsing import compute_firsts, compute_local_first

firsts = compute_firsts(G)
firsts[G.EOF] = ContainerSet(G.EOF)

def expand(item, firsts):
    next_symbol = item.NextSymbol
    if next_symbol is None or not next_symbol.IsNonTerminal:
        return []
    
    lookaheads = ContainerSet()
    # Your code here!!! (Compute lookahead for child items)
    
    assert not lookaheads.contains_epsilon
    # Your code here!!! (Build and return child items)
    return

for x in expand(item, firsts) :
    print(x)
assert str(expand(item, firsts)) == "[A -> .int+A, {'='}, A -> .int, {'='}]"

Como segundo paso, implementaremos la función `compress`. Esta recibe un conjunto de items LR(1) y devuelve el mismo conjunto pero en el que los items con mismo centro están unidos (se combinan los lookahead).

In [ ]:
def compress(items):
    centers = {}

    for item in items:
        center = item.Center()
        try:
            lookaheads = centers[center]
        except KeyError:
            centers[center] = lookaheads = set()
        lookaheads.update(item.lookaheads)
    
    return { Item(x.production, x.pos, set(lookahead)) for x, lookahead in centers.items() }

compress([
    Item(E.productions[0], 0, lookaheads=(G.EOF,)),
    Item(E.productions[0], 0, lookaheads=(plus,)),
    Item(E.productions[0], 1, lookaheads=(plus,)),
    Item(E.productions[0], 2, lookaheads=(plus,)),
    Item(E.productions[0], 2, lookaheads=(plus,G.EOF)),
])

### Clausura

Finalmente implementaremos la función clausura de un conjunto de items LR(1). Recordemos que:

> $CL(I) = I \cup \{ X \rightarrow .\beta, b\}$ tales que:
> - $Y \rightarrow \alpha .X \delta, c \in CL(I)$
> - $b \in First(\delta c)$.

Apoyándonos en las dos funciones anteriores, debería resultar relativamente simple. Como de costumbre para los algoritmos de este tipo, utilizaremos una técnica de punto fijo. 

In [ ]:
def closure_lr1(items, firsts):
    closure = ContainerSet(*items)
    
    changed = True
    while changed:
        changed = False
        
        new_items = ContainerSet()
        # Your code here!!!

        changed = closure.update(new_items)
        
    return compress(closure)

closure = closure_lr1([item, item.NextItem().NextItem()], firsts)
for x in closure:
    print(x)

expected = {
    Item(E.productions[0], 0, lookaheads=(plus, G.EOF)),
    Item(E.productions[0], 2, lookaheads=(plus, G.EOF)),
    Item(A.productions[0], 0, lookaheads=(plus, G.EOF, equal)),
    Item(A.productions[1], 0, lookaheads=(plus, G.EOF, equal)),
}
assert closure == expected

## Goto

Se provee la implementación de la función `goto(Ii, s)`. Recordemos que:

> $Goto(I,X) = CL(\{ Y \rightarrow \alpha X. \beta, c | Y \rightarrow \alpha .X \beta, c \in I \})$

La función recibe como parámetro un conjunto de items y un símbolo, y devuelve el conjunto `goto(items, symbol)`. El método permite setear el parámentro `just_kernel=True` para calcular solamente el conjunto de items kernels en lugar de todo el conjunto de items. En caso contrario, se debe proveer el conjunto con los `firsts` de la gramática, puesto que serán usados al calcular la clausura.

In [ ]:
def goto_lr1(items, symbol, firsts=None, just_kernel=False):
    assert just_kernel or firsts is not None, '`firsts` must be provided if `just_kernel=False`'
    items = frozenset(item.NextItem() for item in items if item.NextSymbol == symbol)
    return items if just_kernel else closure_lr1(items, firsts)

goto = goto_lr1([item], A, firsts)
print(goto)
assert  goto == {
    Item(E.productions[0], 1, lookaheads=(plus, G.EOF))
}

### Construcción del autómata LR(1)

Implementemos el algoritmo para construir el autómata LR(1). Recordemos de conferencia que:
- El estado inicial es la clausura del item **`S' -> .S, $`**.
- Todos los estados son finales.
- Las transiciones ocurren con terminales y no terminales.
- La función de transición está dada por la función **goto**.
    - `f(Ii, c) = Goto(Ii, c)`

_**OJO:** Intente usar solo los items kernel al comparar los estados._

In [ ]:
from cmp.automata import State, multiline_formatter

def build_LR1_automaton(G):
    assert len(G.startSymbol.productions) == 1, 'Grammar must be augmented'
    
    firsts = compute_firsts(G)
    firsts[G.EOF] = ContainerSet(G.EOF)
    
    start_production = G.startSymbol.productions[0]
    start_item = Item(start_production, 0, lookaheads=(G.EOF,))
    start = frozenset([start_item])
    
    closure = closure_lr1(start, firsts)
    automaton = State(frozenset(closure), True)
    
    pending = [ start ]
    visited = { start: automaton }
    
    while pending:
        current = pending.pop()
        current_state = visited[current]
        
        for symbol in G.terminals + G.nonTerminals:
            # Your code here!!! (Get/Build `next_state`)
            
            current_state.add_transition(symbol.Name, next_state)
    
    automaton.set_formatter(multiline_formatter)
    return automaton

Recordemos que este autómata, a pesar de presentar diferencias con el autómata LR(0), continua reconociendo el lenguaje de los prefijos viables de una gramática: cadenas que pueden ocurrir en la pila durante el parseo de una cadena válida.

In [ ]:
automaton = build_LR1_automaton(G.AugmentedGrammar())

assert automaton.recognize('E')
assert automaton.recognize(['A','=','int'])
assert automaton.recognize(['int','+','int','+','A'])

assert not automaton.recognize(['int','+','A','+','int'])
assert not automaton.recognize(['int','=','int'])

automaton

## Parser LR(1) canónico

Reutilizaremos la implementación base de parser Shift-Reduce que terminamos en la clase anterior. Recordemos que esta clase se encarga de todo el algoritmo de parsing, dejando en mano de sus herederos la implementación concreta del método `_build_parsing_table` para llenar la tabla Acción-Goto.

In [ ]:
from cmp.tools.parsing import ShiftReduceParser

### Cómo llena la tabla un parser LR(1)?

- **Sea** "$X \to \alpha .c \omega, s$" un item del estado $I_i$ y $Goto(I_i,c) = I_j$.  
**Entonces** $ACTION[I_i,c] = `S_j`$.

- **Sea** "$X \to \alpha ., s$" un item del estado $I_i$.  
**Entonces** $ACTION[I_i,s] = `R_k`$ (producción `k` es $X \to \alpha$).

- **Sea** $I_i$ el estado que contiene el item "$S' \to S., \$$" ($S'$ distinguido).  
**Entonces** $ACTION[I_i,\$] = `OK`$.

- **Sea** "$X \to \alpha .Y \omega, s$" item del estado $I_i$ y $Goto(I_i,Y) = I_j$.  
**Entonces** $GOTO[I_i,Y] = j$.

In [ ]:
class LR1Parser(ShiftReduceParser):
    def _build_parsing_table(self):
        G = self.G.AugmentedGrammar(True)
        
        automaton = build_LR1_automaton(G)
        for i, node in enumerate(automaton):
            if self.verbose: print(i, '\t', '\n\t '.join(str(x) for x in node.state), '\n')
            node.idx = i

        for node in automaton:
            idx = node.idx
            for item in node.state:
                # Your code here!!!
                # - Fill `self.Action` and `self.Goto` according to `item`)
                # - Feel free to use `self._register(...)`)
                pass
        
    @staticmethod
    def _register(table, key, value):
        assert key not in table or table[key] == value, 'Shift-Reduce or Reduce-Reduce conflict!!!'
        table[key] = value

## Probando

Construyamos un parser LR(1) para la gramática de expresiones.

In [ ]:
parser = LR1Parser(G, verbose=True)

### Tablas

Para visualizar las tablas Action y Goto usaremos la clase `DataFrame` de `pandas`.

In [ ]:
from pandas import DataFrame

def encode_value(value):
    try:
        action, tag = value
        if action == ShiftReduceParser.SHIFT:
            return 'S' + str(tag)
        elif action == ShiftReduceParser.REDUCE:
            return repr(tag)
        elif action ==  ShiftReduceParser.OK:
            return action
        else:
            return value
    except TypeError:
        return value

def table_to_dataframe(table):
    d = {}
    for (state, symbol), value in table.items():
        value = encode_value(value)
        try:
            d[state][symbol] = value
        except KeyError:
            d[state] = { symbol: value }

    return DataFrame.from_dict(d, orient='index', dtype=str)

Recordemos que:

- Debe haber a lo sumo una opción en cada celda.

- Deben aparecer todos los estados (salvo $I_0$) entre **ACTION** y **GOTO**.

- Deben aparecer todas las producciones entre los $R_k$ de **ACTION**.

In [ ]:
display(table_to_dataframe(parser.action))
display(table_to_dataframe(parser.goto))

### Parseando ...

Trabajemos sobre la cadena `int + int = int + int`. Si el parser está correctamente implementado deberíamos obtener una derivación extrema derecha en reverso que parta de la oración y llegue al símbolo distinguido.

In [ ]:
derivation = parser([num, plus, num, equal, num, plus, num, G.EOF])

assert str(derivation) == '[A -> int, A -> int + A, A -> int, A -> int + A, E -> A = A]'

derivation

## Propuestas

- Implemente un generador de parsers **LALR(1)**.
- Complete el pipeline de evaluación.